# Manipulate a function
Tutorial by Jonas Wilfert, Tobias Thummerer

## License
Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar, Jonas Wilfert

Licensed under the MIT license. See [LICENSE](https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
This Julia Package *FMI.jl* is motivated by the use of simulation models in Julia. Here the FMI specification is implemented. FMI (*Functional Mock-up Interface*) is a free standard ([fmi-standard.org](http://fmi-standard.org/)) that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file. The user can thus use simulation models in the form of an FMU (*Functional Mock-up Units*). Besides loading the FMU, the user can also set values for parameters and states and simulate the FMU both as co-simulation and model exchange simulation.

## Introduction to the example
This example shows how to parallelize the computation of an FMU in FMI.jl. We can compute a batch of FMU-evaluations in parallel with different initial settings.
Parallelization can be achieved using multithreading or using multiprocessing. This example shows **multi processing**, check `parallel.ipynb` for multithreading.
Advantage of multithreading is a lower communication overhead as well as lower RAM usage.
However in some cases multiprocessing can be faster as the garbage collector is not shared.


The model used is a one-dimensional spring pendulum with friction. The object-orientated structure of the *SpringFrictionPendulum1D* can be seen in the following graphic.

![svg](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/pics/SpringFrictionPendulum1D.svg?raw=true)  


## Target group
The example is primarily intended for users who work in the field of simulations. The example wants to show how simple it is to use FMUs in Julia.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMI.jl/blob/main/example/distributed.ipynb) there is also a [Julia file](https://github.com/thummeto/FMI.jl/blob/main/example/distributed.jl) with the same name, which contains only the code cells and for the documentation there is a [Markdown file](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/distributed.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | Alternative                                    |   
|:----|:----------------------------------|:--------------------------|:-----------------------------------------------|
| 1.  | Enter Package Manager via         | ]                         |                                                |
| 2.  | Install FMI via                   | add FMI                   | add " https://github.com/ThummeTo/FMI.jl "     |
| 3.  | Install FMIZoo via                | add FMIZoo                | add " https://github.com/ThummeTo/FMIZoo.jl "  |
| 4.  | Install FMICore via               | add FMICore               | add " https://github.com/ThummeTo/FMICore.jl " |
| 5.  | Install BenchmarkTools via        | add BenchmarkTools        |                                                |

## Code section



Adding the desired amount of processes

In [2]:
using Distributed
n_procs = 8
addprocs(n_procs; exeflags=`--project=$(Base.active_project()) --threads=auto`, restrict=false)

8-element Vector{Int64}:
 2
 3
 4
 5
 6
 7
 8
 9

To run the example, the previously installed packages must be included. 

In [3]:
# imports
@everywhere using FMI
@everywhere using FMIZoo
@everywhere using BenchmarkTools

Checking that we workers have been correctly initialized

In [4]:
workers()

@everywhere println("Hello World!")
# The following lines can be uncommented for more advanced informations about the subprocesses
# @everywhere println(pwd())
# @everywhere println(Base.active_project())
# @everywhere println(gethostname())
# @everywhere println(VERSION)
# @everywhere println(Threads.nthreads())

Hello World!
      From worker 7:	Hello World!
      From worker 2:	Hello World!
      From worker 8:	Hello World!
      From worker 4:	Hello World!
      From worker 3:	Hello World!
      From worker 5:	Hello World!
      From worker 9:	Hello World!
      From worker 6:	Hello World!


### Simulation setup

Next, the batch size and input values are defined.

In [5]:

# Best if batchSize is a multiple of the threads/cores
batchSize = 16

# Define an array of arrays randomly
input_values = collect(collect.(eachrow(rand(batchSize,2))))

16-element Vector{Vector{Float64}}:
 [0.3188852284766548, 0.2513698730357842]
 [0.2474388814738956, 0.9469903081415086]
 [0.2137999310869768, 0.48479058597974156]
 [0.37580206796425153, 0.5526608893785356]
 [0.5653639067720981, 0.43058160315153915]
 [0.11538651074939565, 0.7784689811381355]
 [0.9683407952229478, 0.2856696713251521]
 [0.1494844600509394, 0.4514714311819512]
 [0.02596701154934966, 0.3838525863834209]
 [0.6594125630189015, 0.7508628139465704]
 [0.6096880812878464, 0.6095140333451631]
 [0.11550881859291096, 0.09599444449218408]
 [0.9677152093295288, 0.13539276908907594]
 [0.7846602725952101, 0.8398493729401035]
 [0.6168821731089262, 0.867155205806087]
 [0.06987286896343003, 0.5553283588948944]

### Shared Module
For Distributed we need to split of the FMU into a different `module`. This prevents Distributed from trying to serialize and send the FMU over the Network, as this can cause issues. It needs to be made available on all processes using `@everywhere`.

In [ ]:
@everywhere module SharedModule
    using FMIZoo
    using FMI

    t_start = 0.0
    t_step = 0.1
    t_stop = 10.0
    tspan = (t_start, t_stop)
    tData = collect(t_start:t_step:t_stop)

    model_fmu = FMIZoo.fmiLoad("SpringPendulum1D", "Dymola", "2022x")
    FMI.fmiInstantiate!(model_fmu)
end

We define a helper function to calculate the FMU and combine it into an Matrix.

In [7]:
@everywhere function runCalcFormatted(fmu, x0, recordValues=["mass.s", "mass.v"])
    data = fmiSimulateME(fmu, SharedModule.t_start, SharedModule.t_stop; recordValues=recordValues, saveat=SharedModule.tData, x0=x0, showProgress=false, dtmax=1e-4)
    return reduce(hcat, data.states.u)
end

Running a single evaluation is pretty quick, therefore the speed can be better tested with BenchmarkTools.

In [8]:
@benchmark data = runCalcFormatted(SharedModule.model_fmu, rand(2))

BenchmarkTools.Trial: 15 samples with 1 evaluation.
 Range (min … max):  328.156 ms … 351.993 ms  ┊ GC (min … max): 2.14% … 4.35%
 Time  (median):     335.335 ms               ┊ GC (median):    4.17%
 Time  (mean ± σ):   335.519 ms ±   5.623 ms  ┊ GC (mean ± σ):  3.85% ± 0.85%

  █ █     ██ █ █ █  ███ ███      █                            █  
  █▁█▁▁▁▁▁██▁█▁█▁█▁▁███▁███▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  328 ms           Histogram: frequency by time          352 ms <

 Memory estimate: 119.32 MiB, allocs estimate: 3002341.

### Single Threaded Batch Execution
To compute a batch we can collect multiple evaluations. In a single threaded context we can use the same FMU for every call.

In [11]:
println("Single Threaded")
@benchmark collect(runCalcFormatted(SharedModule.model_fmu, i) for i in input_values)

Single Threaded


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 5.411 s (4.01% GC) to evaluate,
 with a memory estimate of 1.86 GiB, over 48037444 allocations.

### Multithreaded Batch Execution
In a multithreaded context we have to provide each thread it's own fmu, as they are not thread safe.
To spread the execution of a function to multiple processes, the function `pmap` can be used.

In [12]:
println("Multi Threaded")
@benchmark pmap(i -> runCalcFormatted(SharedModule.model_fmu, i), input_values)

Multi Threaded


BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  959.444 ms …   1.015 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     999.206 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   994.294 ms ± 21.717 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                    █               █          █         ██  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁██ ▁
  959 ms          Histogram: frequency by time          1.01 s <

 Memory estimate: 77.20 KiB, allocs estimate: 1056.

### Unload FMU

After calculating the data, the FMU is unloaded and all unpacked data on disc is removed.

In [13]:
@everywhere fmiUnload(SharedModule.model_fmu)

### Summary

In this tutorial it is shown how multi processing with `Distributed.jl` can be used to improve the performance for calculating a Batch of FMUs.